In [1]:
import cozmo # pip install --user cozmo[camera]
from PIL import Image
import cv2  # pip install opencv-python
from cozmo.util import degrees, distance_mm, speed_mmps
import random
import sys
import os
import numpy as np
import time

In [ ]:

# Hello world demo code for testing the connection to the Anki Cozmo robot
def cozmo_program(robot: cozmo.robot.Robot):
    robot.say_text("Hello World").wait_for_completed()


cozmo.run_program(cozmo_program)

We now take a sequence of pictures at different angles and store them in a subdirectory. 

In [ ]:
# Adapted from http://cs.gettysburg.edu/~tneller/archive/cs371/cozmo/22sp/fuller/PictureTaking.py
# Todd W. Neller

# Use python versions between 3.5 and 3.9 inclusive. This group used 3.9.7
print("Python version", sys.version)

image_dict = {}

def take_pictures(robot):
  robot.say_text("taking pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True
  latest_image = robot.world.latest_image
  while latest_image is None:
    print('Awaiting first comzo image...')
    time.sleep(1)
    latest_image = robot.world.latest_image
  # Saves picture of what Cozmo sees every incremental number of degrees.
  directory_name = 'images'
  if not os.path.exists(directory_name):
    os.mkdir(directory_name)
  degree_increment = 5
  i = 0
  image_dict = {}
  while i < 400 / degree_increment:
    filename = f'{directory_name}/{i}.jpg'
    latest_image = robot.world.latest_image
    if latest_image is not None:
      annotated = latest_image.annotate_image()
      converted = annotated.convert()
      print(f'saving image {filename}...')
      converted.save(filename, "JPEG", resolution=10)
      image_dict[i] = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2GRAY)
    print(image_dict[i].shape)
    robot.turn_in_place(degrees(degree_increment)).wait_for_completed()
    i += 1

# spins the cozmo more than 360 degrees to get a panorama image of its current environment
def cozmo_program(robot: cozmo.robot.Robot):
  take_pictures(robot)

cozmo.run_program(cozmo_program)


In [ ]:
def take_pictures_turn_sequence(robot):
  robot.say_text("taking random turn pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True
  latest_image = robot.world.latest_image
  while latest_image is None:
    print('Awaiting first comzo image...')
    time.sleep(1)
    latest_image = robot.world.latest_image
  # Saves picture of what Cozmo sees every incremental number of degrees.
  directory_name = 'images'
  if not os.path.exists(directory_name):
    os.mkdir(directory_name)
  i = 0
  while i < 30:
    turn_angle = random.randint(0, 360)
    if turn_angle > 180:
      turn_angle -= 360
    print(f'Turning {turn_angle} degrees...') 
    robot.turn_in_place(degrees(turn_angle)).wait_for_completed()
    filename = f'{directory_name}/turn_{i}_{turn_angle}.jpg'
    latest_image = robot.world.latest_image
    if latest_image is not None:
      annotated = latest_image.annotate_image()
      converted = annotated.convert()
      print(f'saving image {filename}...')
      converted.save(filename, "JPEG", resolution=10)
    i += 1

# spins the cozmo 360 degrees to get a panorama image of its current environment
def cozmo_program(robot: cozmo.robot.Robot):
  take_pictures_turn_sequence(robot)

cozmo.run_program(cozmo_program)